In [1]:
#!/usr/bin/env python
 
import cv2
import numpy as np
import os
import glob
import time
from tqdm import tqdm
import json

# Defining the dimensions of checkerboard
CHECKERBOARD = (9,7)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
 
# Creating vector to store vectors of 3D points for each checkerboard image
objpoints = []
# Creating vector to store vectors of 2D points for each checkerboard image
imgpoints = [] 
 
 
# Defining the world coordinates for 3D points
objp = np.zeros((1, CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
prev_img_shape = None
 
# Extracting path of individual image stored in a given directory
images = glob.glob('/home/hemanth/Desktop/3D CV/camera_calibration/S20/*.jpg')[:]
for fname in tqdm(images):
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # Find the chess board corners
    # If desired number of corners are found in the image then ret = true
    ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)
     
    """
    If desired number of corner are detected,
    we refine the pixel coordinates and display 
    them on the images of checker board
    """
    if ret == True:
        objpoints.append(objp)
        # refining pixel coordinates for given 2d points.
        corners2 = cv2.cornerSubPix(gray, corners, (11,11),(-1,-1), criteria)
         
        imgpoints.append(corners2)
 
        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, CHECKERBOARD, corners2, ret)
    imgs = cv2.resize(img,(900,1600))
    cv2.imshow('img',imgs)
    cv2.waitKey(1)
    
cv2.destroyAllWindows() 
 
h,w = img.shape[:2]
 
"""
Performing camera calibration by 
passing the value of known 3D points (objpoints)
and corresponding pixel coordinates of the 
detected corners (imgpoints)
"""
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
 
print("Camera matrix : \n")
print(mtx)
print("dist : \n")
print(dist)
print("rvecs : \n")
print(rvecs)
print("tvecs : \n")
print(tvecs)
jdict = {}
jdict["mtx"]=mtx
jdict["dist"]=dist
jdict["rvecs"]=rvecs
jdict["tvecs"]=tvecs

100%|█████████████████████████████████████████| 180/180 [02:13<00:00,  1.35it/s]


Camera matrix : 

[[3.12743055e+03 0.00000000e+00 1.13999343e+03]
 [0.00000000e+00 3.12696688e+03 2.04139661e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
dist : 

[[-0.01618344  0.15836294  0.00121001 -0.000984   -0.26485792]]
rvecs : 

(array([[-0.51155879],
       [-0.49691503],
       [ 1.56554495]]), array([[-0.03404132],
       [-0.12350083],
       [ 1.57817991]]), array([[ 0.08808997],
       [-0.31033979],
       [ 1.57862485]]), array([[0.52995333],
       [0.30698933],
       [1.5561577 ]]), array([[-0.31043848],
       [-0.29296186],
       [ 1.63218354]]), array([[0.57029388],
       [0.4693845 ],
       [1.13100058]]), array([[-0.14034902],
       [-0.52893056],
       [ 1.57685018]]), array([[-0.27861078],
       [ 0.49930053],
       [ 1.54566337]]), array([[-0.64575491],
       [-0.79882409],
       [ 1.36188436]]), array([[0.52882791],
       [0.14857638],
       [1.62684359]]), array([[-0.30860828],
       [-0.77253257],
       [ 1.3610071 ]]), array([[ 0.039

In [2]:
'''#data = json.dumps(jdict)
#print(data)
#print(jdict)
data = json.dumps(jdict,indent=4)
print(data)
with open('/home/hemanth/Desktop/3D CV/camera_calibration/S20/cal_results.json','w') as file:
    json.dump(data,file,indent=4)
file.close()'''

"#data = json.dumps(jdict)\n#print(data)\n#print(jdict)\ndata = json.dumps(jdict,indent=4)\nprint(data)\nwith open('/home/hemanth/Desktop/3D CV/camera_calibration/S20/cal_results.json','w') as file:\n    json.dump(data,file,indent=4)\nfile.close()"

In [3]:
'''data = json.loads('/home/hemanth/Desktop/3D CV/camera_calibration/S20/cal_results.json')
print(data)'''

"data = json.loads('/home/hemanth/Desktop/3D CV/camera_calibration/S20/cal_results.json')\nprint(data)"

In [4]:
img = cv2.imread('/home/hemanth/Desktop/3D CV/camera_calibration/S20/20240325_111529.jpg')
h, w = img.shape[:2]
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))
print(newcameramtx,'=====================================')
print(roi)

[[3.10178670e+03 0.00000000e+00 1.13745794e+03]
 [0.00000000e+00 3.10772526e+03 2.04919162e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]] =====================================
(13, 18, 2241, 4000)


In [5]:
# Refining the camera matrix using parameters obtained by calibration
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))
 
# Method 1 to undistort the image
dst = cv2.undistort(img, mtx, dist, None, newcameramtx)
 
# Method 2 to undistort the image
mapx,mapy=cv2.initUndistortRectifyMap(mtx,dist,None,newcameramtx,(w,h),5)
 
dst = cv2.remap(img,mapx,mapy,cv2.INTER_LINEAR)
 
# Displaying the undistorted image
imgs = cv2.resize(dst,(900,1600))

cv2.imshow("undistorted image",imgs)
cv2.waitKey(0)
cv2.destroyAllWindows() 


In [9]:
mean_error = 0
for i in range(len(objpoints)):
 imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
 error = cv2.norm(imgpoints[i], imgpoints2, cv2.NORM_L2)/len(imgpoints2)
 mean_error += error
 
print( "total error: {}".format(mean_error/len(objpoints)) )

total error: 0.21361083437769351
